## How to use 


If you know your Single Business Identifier (SBI) then you can use it to retrieve RPA field boundaries from the Agrimetrics Shapes API.

In order to do this you will also need an API subscription key. If you don't have a subscription key then please refer to [this documention](https://developer.agrimetrics.co.uk/docs/getting-started) to obtain your key.

Once you have your SBI number and your API Key you can call the Shapes API. This is a two-step process:

1. Import your boundaries by providing your SBI to the Shapes API
2. Find out the identifiers for your boundaries by querying the Shapes API
3. Retrieve boundaries by querying the Shapes API

   

### Install dependencies

In [ ]:
import sys
!{sys.executable} -m pip install shapely geopandas matplotlib --quiet

### Set SBI number

In [ ]:
import os
from getpass import getpass
from shapely.geometry import mapping, shape as shapelyShape

if "SBI_KEY" in os.environ:
    sbi = os.environ["SBI_KEY"]
else:
    sbi = getpass(prompt="Enter your SBI:").strip()

### Set API key

In [ ]:
if "API_KEY" in os.environ:
    subscription_key = os.environ["API_KEY"]
else:
    subscription_key = getpass(prompt="Enter your subscription key:").strip()

### Import RPA shapes

In [ ]:
import requests
import json

import_shapes = input(
    "Do you want to import your RPA field boundaries from DEFRA? y/n:"
)

if import_shapes.lower() == "y":

    url = "https://api.agrimetrics.co.uk/shapes/services/rpa/import"

    payload = json.dumps({"sbi": sbi})
    agrimetrics_headers = {
        "ocp-apim-subscription-key": subscription_key,
        "Content-Type": "application/json",
    }

    response = requests.post(url, headers=agrimetrics_headers, data=payload)

    if response.ok:
        message = json.loads(response.text).get("message", "0 shapes created")
        number_of_shapes = int(message.split()[0])
        print(
            f"{number_of_shapes} RPA boundaries imported to the Agrimetrics Shapes API"
        )

These shapes are polygons that represent your field boundaries. They are stored in our system as geometry objects.

### Get RPA boundaries from Agrimetrics Shapes API

In [ ]:
all_shapes = []

max_pages = 10
results_per_page = 100  # note there is a maximum page size of 100

headers = agrimetrics_headers


def get_shapes(page=1, results=results_per_page):
    shapes_url = f"https://api.agrimetrics.co.uk/shapes/shapes?pageNum={page}&pageSize={results}"
    shapes_response = requests.get(shapes_url, headers=agrimetrics_headers).json()
    print(shapes_response)
    return shapes_response["shapes"]


for page in range(1, max_pages + 1):
    shapes = get_shapes(page)
    if not shapes:
        break
    all_shapes.extend(shapes)

rpa_shapes = [shape for shape in all_shapes if shape["source"] == "rpa"]

### Retrieve individual polygons as a GeoJson collection

In [ ]:
urls=[shape['id'] for shape in rpa_shapes]

def get_polygons(shapes_url):
    shapes_response = requests.get(shapes_url, headers=agrimetrics_headers).json()
    return shapes_response

def turn_2_feature(polygon):
    return {
        "type": "Feature",
        "properties": {},
        "geometry": polygon
    }
polygons = [turn_2_feature(get_polygons(url)['boundary']) for url in urls]
with open('polygons.json', 'w')as f:
    json.dump(polygons, f, indent=2)